In [2]:
import pandas as pd 
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.layers import Dropout
import xgboost
import pickle

In [3]:
alpha_key = 'RNZPXZ6Q9FEFMEHM'
ticker = 'MSFT'
periods = 20
csvPath = './DATA/'+ ticker +'.csv'
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key=alpha_key, output_format='pandas')

data = ts.get_intraday(symbol=ticker,interval='15min', outputsize='full')
data = data[0]
data.rename(lambda x: x[2:].strip(), axis='columns', inplace=True)
data

from alpha_vantage.techindicators import TechIndicators
ti = TechIndicators(key=alpha_key, output_format='pandas')

indicator_roc = ti.get_roc(symbol=ticker, interval='15min', time_period=periods)
indicator_roc = indicator_roc[0]


indicator_sma = ti.get_sma(symbol=ticker, interval='15min', time_period=periods)
indicator_sma = indicator_sma[0]


indicator_rsi = ti.get_rsi(symbol=ticker, interval='15min', time_period=periods)
indicator_rsi = indicator_rsi[0]


def BollingerBand(data, n_loockback, n_std=2):
    hlc_avg = (data.high + data.low + data.close)/3
    data_std = hlc_avg.rolling(window=n_loockback).std()
    data_upper = data.SMA + n_std * data_std
    data_lower = data.SMA - n_std * data_std
    return data.SMA, data_lower, data_upper

indicator_bb = ti.get_bbands(symbol=ticker, interval='15min', time_period=periods)
indicator_bb = indicator_bb[0]


original_df = pd.merge(data, indicator_roc, on='date', how='inner')
original_df = pd.merge(original_df, indicator_sma, on='date', how='inner')
original_df = pd.merge(original_df, indicator_rsi, on='date', how='inner')
original_df = pd.merge(original_df, indicator_bb, on='date', how='inner')
original_df["Real Middle Band"], original_df["Real Lower Band"], original_df["Real Upper Band"] = BollingerBand(original_df, 20, 2)
original_df = original_df.dropna()
original_df = original_df.iloc[::-1]
original_df.to_csv(csvPath)